In [1]:
import pandas as pd
import numpy as np
import random
import re
import torch
from tqdm import tqdm

In [2]:
df = pd.read_csv("Translation_triplets.csv")
df.dropna(inplace = True)
df

topic  paper_ID  sentence_ID          info-unit  \
0            Blogs         0          269  ablation-analysis   
1            Blogs         0          269  ablation-analysis   
2            Blogs         0          269  ablation-analysis   
3            Blogs         0          269  ablation-analysis   
4            Blogs         0          317  ablation-analysis   
...            ...       ...          ...                ...   
54553  translation       867          157            results   
54554  translation       867          157            results   
54555  translation       867          157            results   
54556  translation       867          157            results   
54557  translation       867          157            results   

                     sub           pred                      obj  pred_weights  
0       coverage penalty            are           less effective      0.576998  
1         less effective            for                   models      0.642904  
2                 models           with            RL refinement      0.626849  
3      Ablation analysis      find that     length normalization      0.610668  
4                  model  first suffers                   little      0.814908  
...                  ...            ...                      ...           ...  
54553    proposed method       achieves               advantages      0.616635  
54554         advantages           over            jointtraining      0.685659  
54555      jointtraining             in        online adaptation      0.531285  
54556      jointtraining             in  translation performance      0.490170  
54557            Results            has          proposed method      0.584522  

[54558 rows x 8 columns]

In [3]:
df_temp = df.rename(columns = {"sub" : "obj", "obj" : "sub"})
df_temp

topic  paper_ID  sentence_ID          info-unit  \
0            Blogs         0          269  ablation-analysis   
1            Blogs         0          269  ablation-analysis   
2            Blogs         0          269  ablation-analysis   
3            Blogs         0          269  ablation-analysis   
4            Blogs         0          317  ablation-analysis   
...            ...       ...          ...                ...   
54553  translation       867          157            results   
54554  translation       867          157            results   
54555  translation       867          157            results   
54556  translation       867          157            results   
54557  translation       867          157            results   

                     obj           pred                      sub  pred_weights  
0       coverage penalty            are           less effective      0.576998  
1         less effective            for                   models      0.642904  
2                 models           with            RL refinement      0.626849  
3      Ablation analysis      find that     length normalization      0.610668  
4                  model  first suffers                   little      0.814908  
...                  ...            ...                      ...           ...  
54553    proposed method       achieves               advantages      0.616635  
54554         advantages           over            jointtraining      0.685659  
54555      jointtraining             in        online adaptation      0.531285  
54556      jointtraining             in  translation performance      0.490170  
54557            Results            has          proposed method      0.584522  

[54558 rows x 8 columns]

In [4]:
df = pd.concat([df, df_temp])
df

topic  paper_ID  sentence_ID          info-unit  \
0            Blogs         0          269  ablation-analysis   
1            Blogs         0          269  ablation-analysis   
2            Blogs         0          269  ablation-analysis   
3            Blogs         0          269  ablation-analysis   
4            Blogs         0          317  ablation-analysis   
...            ...       ...          ...                ...   
54553  translation       867          157            results   
54554  translation       867          157            results   
54555  translation       867          157            results   
54556  translation       867          157            results   
54557  translation       867          157            results   

                           sub           pred                   obj  \
0             coverage penalty            are        less effective   
1               less effective            for                models   
2                       models           with         RL refinement   
3            Ablation analysis      find that  length normalization   
4                        model  first suffers                little   
...                        ...            ...                   ...   
54553               advantages       achieves       proposed method   
54554            jointtraining           over            advantages   
54555        online adaptation             in         jointtraining   
54556  translation performance             in         jointtraining   
54557          proposed method            has               Results   

       pred_weights  
0          0.576998  
1          0.642904  
2          0.626849  
3          0.610668  
4          0.814908  
...             ...  
54553      0.616635  
54554      0.685659  
54555      0.531285  
54556      0.490170  
54557      0.584522  

[109116 rows x 8 columns]

In [5]:
predicate = df['pred'].to_list()

In [6]:
def remove(x):
    return x.replace('"', ' ').lower()

In [7]:
df['sub'] = df['sub'].apply(remove)
df['obj'] = df['obj'].apply(remove)

In [8]:
sub_obj = pd.concat([df['sub'], df['obj']]).unique()
print(len(sub_obj))

31926


In [9]:
nodes = df['sub'].unique()
print(len(nodes))

31926


In [10]:
df['pred_weights'].mean()

0.6217118123449078

In [11]:
tr_nodes = {phrase : 1 for phrase in nodes}
tr_updated_nodes = {phrase : 1 for phrase in nodes}

In [12]:
d = 0.85**6
print(d)

0.37714951562499993


In [13]:
df['triplet'] = df['sub'] + " " + df['pred'] + " " + df['obj']
df

topic  paper_ID  sentence_ID          info-unit  \
0            Blogs         0          269  ablation-analysis   
1            Blogs         0          269  ablation-analysis   
2            Blogs         0          269  ablation-analysis   
3            Blogs         0          269  ablation-analysis   
4            Blogs         0          317  ablation-analysis   
...            ...       ...          ...                ...   
54553  translation       867          157            results   
54554  translation       867          157            results   
54555  translation       867          157            results   
54556  translation       867          157            results   
54557  translation       867          157            results   

                           sub           pred                   obj  \
0             coverage penalty            are        less effective   
1               less effective            for                models   
2                       models           with         rl refinement   
3            ablation analysis      find that  length normalization   
4                        model  first suffers                little   
...                        ...            ...                   ...   
54553               advantages       achieves       proposed method   
54554            jointtraining           over            advantages   
54555        online adaptation             in         jointtraining   
54556  translation performance             in         jointtraining   
54557          proposed method            has               results   

       pred_weights                                           triplet  
0          0.576998               coverage penalty are less effective  
1          0.642904                         less effective for models  
2          0.626849                         models with rl refinement  
3          0.610668  ablation analysis find that length normalization  
4          0.814908                        model first suffers little  
...             ...                                               ...  
54553      0.616635               advantages achieves proposed method  
54554      0.685659                     jointtraining over advantages  
54555      0.531285                online adaptation in jointtraining  
54556      0.490170          translation performance in jointtraining  
54557      0.584522                       proposed method has results  

[109116 rows x 9 columns]

In [14]:
pred_dict = dict(zip(df.triplet, df.pred_weights))
print(len(pred_dict), len(df.index))

100712 109116


In [16]:
for i in range(3,6):
    for sub in nodes:
        df_temp = df[df['sub'] == sub]
        x = df_temp['pred'].to_list()
        y = df_temp['obj'].to_list()
        summation = 0
        for pred, obj in zip(x,y):
            z = df_temp[(df_temp['obj'] == obj)]['triplet'].to_list()
            sum_den = sum([pred_dict[key] for key in z])
            if(sum_den == 0):
                continue
            triplet = sub + " " + pred + " " + obj
            add = pred_dict[triplet] * tr_nodes[obj] / sum_den
            summation += add
        print(summation)
        tr_updated_nodes[sub] = 1-d + d*summation
    for sub in nodes:
        tr_nodes[sub] = tr_updated_nodes[sub]
    df_nodes = pd.DataFrame(tr_nodes.items(), columns = ['node', 'tr_score'])
    df_nodes.to_csv(f"Translation_TR_Score/TR_Score_{i+1}.csv", index = None)

465730.3264041784
69946.48076725329
1885017.834869947
2803702.697801065
8002709.323439795
491297.8329802363
422222.7052305563
873997.9142613129
75078.39424053961
3038.0196552628986
58675.58739673336
2136171.443215839
7625.330397164704
9594.003652203653
603215.4308848727
2792163.1905988
2.2844835142723436
1706735.2814544686
28726.6822080786
128558.4858735638
1516.7198096650025
3581.910537953407
204.3840595908735
1711538.3825228373
117990.29385025943
117990.29385025943
1.0
138193.1018520524
1330737.5861194239
12845.411226584085
663.1488252377281
992262.8605571156
155824.514197542
2281589.3747472605
218149.6646754747
410068.71532685054
1534530.774459457
123140.26216273035
691517.3023062194
820836.8627591868
2982.5567458520554
555521.4369497142
2802.049839216562
22560.12595128399
1.0
1.0
427385.9964326013
6951.480218030226
950.4036869551273
434221.0497960623
1284855.8207648527
422673.8324320593
10424.735332639724
561205.0122216386
1698588.3498026133
15154.97203382693
147870.92982640615
435

137282.44801090675
8661.151137860492
28930.96499392437
1403205.181532714
104335.34458593384
735220.897685833
592725.9115407808
761584.2461435817
1335742.7562243906
576.9889264879632
5913.332343664295
14731.846073802411
474875.5232309696
4163.746751009237
177.25546474053078
1777.6372059933817
33516.890593104705
4085.8233711333733
1673.6156095935776
847570.649482593
730366.3706118297
569508.2865320616
258398.98979164805
26113.54020388028
121129.57991656387
239478.34229000326
486.36569332224997
120557.23572463717
897.9266530132372
117990.77829259467
807.1844049359265
117990.29385025943
3.8534505428170323
3435.0023518516546
384.57571694565024
12210.986750406744
10346.181304173258
389.65530817102285
878356.3330683119
1149474.220945157
4158.033434027388
1.0
14749.25395286778
810676.1945787661
14239.084655371611
91840.15940685786
191.04689976060612
849310.573345097
5250.791435288203
2660.6230936145876
797230.4467681022
78438.65585144646
4088.804560518489
726926.4063084411
1982.4623414165535
7

412.12341907536427
10731.997304388116
10.945090759978612
18.233108534536292
15.048660193568
7.181718838659147
422361.9138123206
726926.5322987329
2032.4729761665549
1384.4875708794211
735562.4590663901
728360.3204970076
726944.647712422
2842.4990269009663
727117.6557593979
4306.504459311457
727141.579831137
1.6266840923722843
2.822572259314012
2.822572259314012
4528.526352476216
424368.23815821024
443118.5577813177
422359.02156115917
427735.00828047236
10025.442310573915
1868.516420052222
139.0820071583909
564062.2983659866
1207808.5045669766
2056.19559972753
45.44216519425713
27655.724315671647
729426.3480397067
51990.20850322028
218.48758904163788
3290.4073755271443
36950.21314674351
4239.374199615668
2240.980234245323
22335.04758057924
509.5089112247273
118049.01494120316
120354.32628061771
737188.2292692133
117990.29385025943
745446.0939598179
2363.49300724019
1307374.340302429
1946.455620208679
81992.31178424576
436476.3009675257
754938.440804973
4904.305977173722
2374.36466288730

43894.75929488584
35034.19080203476
63107.87396980311
781315.0181948377
729683.2729109279
375.18060578024006
5.674745423145253
17201.049400637447
7855.063652152645
2262.152537299648
89.70616665796298
726926.4250059134
1983.1020276801985
7558.311886053548
7589.934544275152
4840.746955774551
411.3009452639652
41810.68929954671
2480.0282907972755
79754.75180612219
867347.8635636943
2260.150328338637
1227899.554593162
36937.76566951795
977.4981254290991
79826.3921598529
154124.64304613456
1786.58997623767
890382.6100291377
422510.9619811136
6141.452789319753
17816.00840484329
16967.453195238744
8276.713089427069
8279.854875165269
765566.3679766752
33621.927117913336
726925.530089772
1004060.8616303377
48970.93621125596
8777.198505163808
223117.49116961376
11842.066512695017
4374.0601417467215
727303.4556301157
1983.1060983342206
2228.9604212055474
33456.08156811115
422783.0361018562
14775.104042323024
7959.177221465952
460284.2849724824
1.0
728799.4441051247
114593.34187852847
756303.96228

736.2337795373169
82871.92750682287
737.5906069336752
7929.014672297852
25181.950992722257
731196.3568029528
727629.3230698062
81946.65219168864
9483.599961584194
807596.3726502362
82650.44517172295
69246.18546308948
792.0239040711567
495068.4313304222
519409.95633000036
575827.1779976405
73116.31317032082
125555.94927578271
246314.64144476966
45102.1594422414
122850.78911614648
8880.838308779548
118063.31226008185
3496.3171284487325
124605.89371333529
135021.90236354974
10688.51678733872
237371.52501520273
25801.926545142698
3121.601308216372
475461.2656374983
73429.1423458181
423032.6588503296
16897.102134560006
427207.2514576444
426987.48734823446
429988.33666131063
25801.926545142698
422222.7052305563
1286889.4580776857
1.0
445986.57858193736
25145.451230820363
423445.27586098353
6683.795238488693
23201.382330222623
14630.477000423341
1541766.3542340687
20251.768675699255
80024.88290031915
1175238.3267126745
57054.440636032414
512537.67423915205
422222.7052305563
429383.1343115966


...

186018.86558288062
103426.86852252927
42.69774676401213
24639936.513308905
1.7255360427657886
1.7255360427657889
1.0
239.84122402730546
66402.14657151214
7992048.258013093
360571.18168693397
319265.0834855331
60159.83401426356
60418.91391218658
60159.83401426355
60159.83401426356
1138558.0372903012
60159.834014263564
24639936.51330891
1835202.2763833874
1835202.2763833874
2260311.148495638
465648.5903455406
4377044.976551983
10801804.413374042
4665345.097472126
4373442.18990048
19967464.8062264
2260311.148495638
319265.0834855331
356.7781555970443
113345.01354063499
3110119.209506304
302410.08745510585
124977.43554690796
1269033.8317643728
11634.122367770953
11634.122367770953
76545.24815973554
2390490.9645163203
614.9627740156754
614.9627740156754
3847824.4679848747
10047840.164053965
227309.42030217868
12061833.403706362
12061833.403706362
391797633.35111785
9601424.89032376
135929.31476623518
7174364.542362079
391797633.35111785
391797633.35111785
113345.01354063499
2689848.02446276

24639936.513308905
1.7255360427657889
391797633.35111785
619.9122117069568
121269.84346171774
329596.0448200224
14646590.680165524
230622.46123661092
1.51496537361527
29888023.054595344
60518.855480015
31.12291576376875
19967464.8062264
303946.8558651399
638566.5068033893
1.0
346126.55223012966
324640.8509250313
2193741.6505784723
1705355.228964253
14757366.31147593
397764.03842977656
1139902.5528277599
1507156.489406072
63324.690119793006
63324.690119793
1199333.048587854
1199333.048587854
24639936.51330891
24639936.513308905
24639936.513308905
24639936.513308905
10811187.321664348
6016835.815728476
212691.8709518009
1.51496537361527
831688.781231608
1596014.179485163
104758.08015802561
104758.08015802561
1.51496537361527
7164.2463162240765
540709.8082364266
739675.299238468
4171.873299342131
108112.33246284504
1147432.1535153692
2794.1593927357094
6095948.659873573
30536.32534486413
180129.80459213248
1975177.6394431535
24639936.513308905
24639936.5133089
32910.98170319413
14757366.3

60098.14818582719
1.0
91465.77809766495
24639936.513308905
1595975.1263838743
103453.41018499873
1990324.2381521165
107375.19690414725
318734.3889025923
1598901.0887557596
1273.7187678240884
601906.050544693
710.7666766714276
24639936.51330891
103426.86852252927
3699.617814011344
3699.617814011344
436494.7062513189
436494.7062513189
48001.46171999567
48001.46171999567
28421.719141831767
1199130.9508790318
60394.98554855791
21.260625959014206
171676.23334089827
171676.23334089827
171676.23334089827
171676.23334089827
1138558.0372903012
60394.98554855791
54934182.61976071
9601424.89032376
171206.64283468184
211.5884768160141
3222643.6161100366
97840.61373890555
553608.3910806802
397525.25260088494
1138558.0372903012
1723509.9209182144
13771047.852543605
1.0
622718.4207976434
1138558.0372903012
24639936.513308905
1479372.7044873603
2142625.711748912
549374.2395973749
105899.04655634728
116.41030753766998
310237.2629679492
858.5287798676203
14757366.31147593
519259.2065031428
5256380.90757

9621.982587499084
950978.4191345779
19167105.44676216
17420772.58162884
1110721.1278602872
527989.1860507204
1906709.9805746337
115695.05833610859
9861.280391652195
9861.280391652195
398917.15942250815
521306.335358249
7992048.258013094
7992048.258013093
21281.252880510376
397277.9160978545
3342540.9298770605
3342540.9298770605
24639936.513308905
103507.29182090067
206987.6665961541
16589846.731532529
16589846.731532529
2580571.093249228
554188.9677914333
3342540.9298770605
115953.79538707268
1.0
19967464.8062264
319637.3061047347
319637.3061047347
319637.3061047347
7505468.872812455
14757366.31147593
319265.0834855331
605626.333035501
1459171.0351282633
178220.53928644382
319367.6031092393
103.51962370622554
970672.708878306
170074.92565281142
1.51496537361527
391797633.35111785
999428.1873696736
377229.5662217084
1872617.110272417
997252.0487556404
242568.73464071882
209330.16341360655
1138558.0372903012
125196.92666130236
7321121.904212543
1595692.938043453
391797633.35111785
421950

24639936.513308905
24639936.51330891
2135.5624845583525
538828.8417317817
38811828.76650155
2005.0091756171082
103426.86852252926
469922.55779457866
103426.86852252927
3450237.5838198615
1492.6377618222245
24639936.513308905
19967464.8062264
1223089.3691389423
303946.8558651399
19967464.8062264
324640.8509250313
115108.67766905276
2056234.9575823501
190383751.80103147
24639936.513308905
24639936.513308905
388849.37165431597
16597.306495260997
116790.52813171054
388849.37165431597
3350090.220503396
3350090.220503396
60078.26091921842
1.0
1.51496537361527
3887080.249128253
1297035.1237590332
1297035.1237590332
61332.53397676725
61332.53397676724
61332.53397676724
9855.792368398805
6017791.86142138
4.320345124843373
5504358.7480734605
190383751.80103147
43450570.843263276
560146.1378775344
18341.88723283552
2973838.8991998145
24639936.513308905
24639936.513308905
3152026.1788684176
106815.67596701352
106815.67596701352
1556.1227566066314
117607.42938310899
121336.99755673355
514284.152634

In [ ]:
print(tr_nodes)